<a href="https://colab.research.google.com/github/caiocesarcosta/detection_transfer_learning_yolo_porject4/blob/main/detection_transfer_learning_yolo_porject4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import json
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import shutil
import yaml
import tensorflow as tf
from google.colab import drive

class YoloImageManagement:
    def __init__(self, num_images_to_download: int, category_names: list):
        self.num_images_to_download = num_images_to_download
        self.category_names = category_names
        self.data_dir = '/content/coco_yolo'
        self.train_dir = os.path.join(self.data_dir, 'images/train')
        self.labels_dir = os.path.join(self.data_dir, 'labels/train')
        self.img_width = 150

        self.mountdriveGoogle()
        self.createDiretory()
        self.image_ids = []
        self.images_data = []
        self.coco_data = self.downloadOrOpenAnnotations()  # carrega o arquivo de anotações

        if self.coco_data: # Previna erros caso as anotações não sejam carregadas corretamente.
            self.category_ids, self.images_data = self.mapImageIDsToCategoryIDs()
            self.loadYoloDataset()
        else:
            print("Falha no download ou leitura das anotações. O processo será interrompido")

    def mountdriveGoogle(self):
        """Monta o Google Drive."""
        try:
            drive.mount('/content/drive', force_remount=True)
            print("Google Drive montado com sucesso!")
        except Exception as e:
            print(f"Erro ao montar o Google Drive: {e}")

    def createDiretory(self):
        """Cria os diretórios necessários."""
        os.makedirs(self.train_dir, exist_ok=True)
        os.makedirs(self.labels_dir, exist_ok=True)
        print(f"Diretórios criados: {self.train_dir}, {self.labels_dir}")

    def convertBboxToYolo(self, bbox, img_width, img_height):
        """Converte bounding box do formato COCO para YOLO."""
        x, y, width, height = bbox
        x_center = ((x + (x + width)) / 2) / img_width
        y_center = ((y + (y + height)) / 2) / img_height
        width_norm = width / img_width
        height_norm = height / img_height
        return x_center, y_center, width_norm, height_norm

    def loadYoloDataset(self):
       """Função para baixar e salvar imagens com bounding boxes no formato YOLO."""
       try:
          # Itera sobre as imagens
          for image_data in self.images_data[:self.num_images_to_download]:
              img_id = image_data['id']
              file_name = image_data['file_name']
              img_width = image_data['width']
              img_height = image_data['height']

              # Construir o arquivo de anotação (YOLO)
              annotation_file = os.path.join(self.labels_dir, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

              # Baixar as imagens do site
              image_url = image_data['coco_url']
              try:
                  response = requests.get(image_url)
                  img = Image.open(BytesIO(response.content))
                  img.save(os.path.join(self.train_dir, file_name))
                  print(f"Imagem {file_name} salva em {self.train_dir}")

                  annotation_content = []
                  for annotation in self.coco_data['annotations']:
                      if annotation['image_id'] == img_id and annotation['category_id'] in self.category_ids:
                          category_name = next((cat['name'] for cat in self.coco_data['categories'] if cat['id'] == annotation['category_id']), None) #Busca o nome da categoria pelo id
                          if category_name:
                              category_index = self.category_names.index(category_name)
                              bbox = annotation['bbox']
                              x_center, y_center, width_norm, height_norm = self.convertBboxToYolo(bbox, img_width, img_height)
                              annotation_content.append(f"{category_index} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

                  # Verifica se gerou o conteudo
                  if annotation_content:
                      print(f"Anotações geradas para {file_name}: {annotation_content}")
                      # Escreve no arquivo
                      with open(annotation_file, 'w') as f:
                          f.write('\n'.join(annotation_content))
                      print(f"Arquivo de anotação {annotation_file} criado.")
                  else:
                      print(f"Nenhuma anotação encontrada para {file_name}")

              except Exception as e:
                  print(f"Erro ao processar a imagem {file_name}: {e}")

       except Exception as e:
          print(f"Erro no LoadYoloDataset: {e}")

    print("Imagens e anotações baixadas e convertidas com sucesso!")


    def downloadOrOpenAnnotations(self):
        """Baixar ou abre as informações da Annotation """
        annotations_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
        annotations_zip_path = os.path.join(self.data_dir, 'annotations_trainval2017.zip')
        annotations_dir = os.path.join(self.data_dir, 'annotations')
        instances_path = os.path.join(annotations_dir, 'instances_train2017.json')

        print(f"Pasta de anotações: {annotations_dir}")
        print(f"Arquivo zip de anotações: {annotations_zip_path}")
        print(f"Arquivo JSON de anotações: {instances_path}")


        # Crie o diretório de anotações, se não existir
        if not os.path.exists(annotations_dir):
            print("Criando pasta para as anotações: " + annotations_dir)
            os.makedirs(annotations_dir, exist_ok=True)

        # Baixa e extrai as anotações, se ainda não existirem
        if not os.path.exists(annotations_zip_path):
            print("Baixando anotações...")
            !wget -q "$annotations_url" -O "$annotations_zip_path"
            print("Extraindo anotações...")
            !unzip -q "$annotations_zip_path" -d "{self.data_dir}"

        try:
            if not os.path.exists(instances_path): # verifica a existencia do arquivo
              print(f"Arquivo de anotação não encontrado em: {instances_path}")
              print("Por favor, verifique se o arquivo foi extraído corretamente.")
              return None

            with open(instances_path, 'r') as f:
                coco_data = json.load(f)
                print("Arquivo de anotações carregado com sucesso.")
                return coco_data
        except Exception as e:
            print(f"Erro ao abrir arquivo de anotação: {e}")
            return None

    def mapImageIDsToCategoryIDs(self):
        """Mapear IDs de imagens para IDs de categorias"""
        if self.coco_data is None:
            print("Erro: coco_data não foi inicializado corretamente.")
            return

        image_to_categories = {}
        for ann in self.coco_data['annotations']:
            image_id = ann['image_id']
            category_id = ann['category_id']
            if image_id not in image_to_categories:
                image_to_categories[image_id] = set() # Usando um set para ter elementos únicos
            image_to_categories[image_id].add(category_id)

        category_ids = [cat['id'] for cat in self.coco_data['categories'] if cat['name'] in self.category_names]
        images_data = [img for img in self.coco_data['images'] if img['id'] in image_to_categories and any(cat_id in image_to_categories[img['id']] for cat_id in category_ids)]
        print ("Arquivos extraidos com sucesso")
        return category_ids, images_data

# ***************************************************************************************************
# Classe TransferLearningYolo
# ***************************************************************************************************
class TransferLearningYolo:
    def __init__(self, coco_dataset: YoloImageManagement):
         self.repoYoloGitHub = 'https://github.com/ultralytics/yolov5'
         self.pretrained_weights = 'yolov5s.pt' # Usa o modelo pequeno (mais rápido)
         self.img_width = coco_dataset.img_width
         self.batch_size = 16
         self.epochs = 40
         self.cocoFile = coco_dataset
         self.data = None

         self.category_names = coco_dataset.category_names
         self.num_classes = len(self.category_names) # Número de classes

         self.cloneRepoistoryYoloGitHub()
         self.installDependency()
         self.createYAMLConfigurationFileToDataset()


    def cloneRepoistoryYoloGitHub(self):
      """Clone the yolo repository, from gitHub."""
      print ("Clonando yolo do github")
      clone_command = f"git clone {self.repoYoloGitHub}"
      os.system(clone_command)  # Use os.system para executar o comando

    def installDependency(self):
      """Install the dependecies."""
      print ("Instalando dependências")
      os.chdir('yolov5')
      !pip install -r requirements.txt

    def createYAMLConfigurationFileToDataset(self):
        """Creates the yaml file to persist the information."""
        print ("Criando arquivo de configuração")
        data = {
           'train': '/content/coco_yolo/images/train',
           'val': '/content/coco_yolo/images/train',  # Use os mesmos dados para validação (para simplificar)
           'nc': self.num_classes,  # Número de classes
           'names': self.category_names  # Nomes das classes
        }

        with open('coco.yaml', 'w') as f:
           yaml.dump(data, f)

         # Imprimi o conteudo do arquivo.
        print("\nConteúdo do arquivo YAML:")
        !cat coco.yaml

    def trainModelYolo(self):
        """Trains the Yolo model."""
        print ("Treinando o modelo yolo")
        train_command = f"python train.py --img {self.img_width} --batch {self.batch_size} --epochs {self.epochs} --data coco.yaml --weights {self.pretrained_weights} --cache"
        os.system(train_command)  # Use os.system

    def detectObjectImage(self):
        """Detect in the test image if the training was executed correctly. """
        print ("Detectando imagem do modelo treinado")
        test_image = '/content/yolov5/data/images/bus.jpg' #imagem de exemplo do yolo
        detect_command = f"python detect.py --weights runs/train/exp/weights/best.pt --img {self.img_width} --conf 0.5 --source \"{test_image}\""
        os.system(detect_command)  # Use os.system

    def train(self):
        """Orchestrates all the functions in the Train process."""
        self.trainModelYolo()
        self.detectObjectImage()

def main():
    num_images = 100  # quantidade de imagens
    classes = ['person', 'dog', 'cat', 'car', 'bicycle', 'motorbike', 'traffic light', 'bird', 'truck']
    coco_dataset = YoloImageManagement(num_images, classes)

    # Check if coco_dataset was initialized successfully
    if coco_dataset.coco_data is not None:
        transferLearningYolo = TransferLearningYolo(coco_dataset)
        transferLearningYolo.train()
        print("Modelo treinado")
    else:
        print("Falha ao inicializar coco_dataset. Verifique se as dependências foram instaladas e o arquivo de anotações está correto.")

if __name__ == "__main__":
  main()

Imagens e anotações baixadas e convertidas com sucesso!
Mounted at /content/drive
Google Drive montado com sucesso!
Diretórios criados: /content/coco_yolo/images/train, /content/coco_yolo/labels/train
Pasta de anotações: /content/coco_yolo/annotations
Arquivo zip de anotações: /content/coco_yolo/annotations_trainval2017.zip
Arquivo JSON de anotações: /content/coco_yolo/annotations/instances_train2017.json
Arquivo de anotações carregado com sucesso.
Arquivos extraidos com sucesso
Imagem 000000391895.jpg salva em /content/coco_yolo/images/train
Anotações geradas para 000000391895.jpg: ['0 0.651281 0.479236 0.240437 0.835361', '0 0.765000 0.546861 0.056125 0.133611', '4 0.783320 0.557778 0.047859 0.097167']
Arquivo de anotação /content/coco_yolo/labels/train/000000391895.txt criado.
Imagem 000000522418.jpg salva em /content/coco_yolo/images/train
Anotações geradas para 000000522418.jpg: ['0 0.798250 0.494073 0.401250 0.988146']
Arquivo de anotação /content/coco_yolo/labels/train/000000522